<a href="https://colab.research.google.com/github/DeokwonWang/Content_Recommend_algorithm/blob/main/movie_recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
pip install surprise

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [70]:
# 데이터 준비
md = pd.read_csv('/content/drive/MyDrive/moviedata/movies_metadata.csv')
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')

# **Content Based Filtering**

In [3]:
links_small = pd.read_csv('/content/drive/MyDrive/moviedata/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [9]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [10]:
# 영화 설명/태그 라인 기반 추천자
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [11]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [12]:
tfidf_matrix.shape

(9099, 268124)

In [13]:
#코사인 유사성을 사용한 두 영화간의 유사성 수치 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [15]:
# 코사인 유사성 점수를 기반으로 가장 유사한 영화 30편 반환
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [16]:
def get_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:31]
  movie_indices = [i[0] for i in sim_scores]
  return titles.iloc[movie_indices]

In [17]:
# Content_Based Filtering 결과값 출력
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [18]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

Dark Knight의 경우 시스템이 배트맨 영화로 식별하고 이후에 다른 배트맨 영화를 최고의 권장 사항으로 추천 할 수 있습니다. 하지만 안타깝게도 이 시스템이 현재 할 수 있는 전부입니다. 

이것은 영화의 등급과 인기를 결정하는 출연진, 제작진, 감독 및 장르와 같은 매우 중요한 기능을 고려하지 않기 때문에 대부분의 사람들에게 별로 유용하지 않습니다. 

Dark Knight를 좋아하는 사람은 Nolan 때문에 더 좋아하고 Batman Forever와 Batman Franchise의 다른 모든 표준 이하 영화를 싫어할 것입니다.

따라서 *개요 및 태그 라인보다 훨씬 더 암시적인 메타 데이터를 사용*할 것입니다. 다음 하위 섹션에서는 장르, 키워드, 출연진 및 제작진을 고려하는보다 정교한 추천자를 구축 할 것입니다.

# **Metadata Based Recommender**

In [19]:
# 현재 Dataset을 크루&키워드 Dataset과 병합하기

credits = pd.read_csv('/content/drive/MyDrive/moviedata/credits.csv')
keywords = pd.read_csv('/content/drive/MyDrive/moviedata/keywords.csv')

In [20]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [21]:
md.shape

(45463, 25)

In [22]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [23]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [24]:
#Crew에서는 감독을, Cast에서는 상위 3명의 배우만 추출

smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['cast'].apply(lambda x: len(x))

In [25]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [26]:
smd['director'] = smd['crew'].apply(get_director)

In [27]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [28]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [29]:
# Genre & Credit 데이터 준비 - 모든 공백 제거(혼동방지), 감독 3번 언급(비중 높이기)

smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [30]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x,x])

In [31]:
# Keywords - 키워드 빈도수 계산

s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [32]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [33]:
# 한번만 발생하는 키워드 제거
s = s[s > 1]

In [34]:
# 비슷한 단어 제거(ex/ dog, dogs)

stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [35]:
def filter_keywords(x):
  words = []
  for i in x:
    if i in s:
      words.append(i)
  return words

In [36]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [37]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: " ".join(x))

In [38]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [39]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [40]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [41]:
# 앞서 사용했던 get_recommendations 함수 재사용(코사인 유사도 변경으로 더 나은 결과)

get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [42]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

**인기도와 평점**

유사성 점수를 기준으로 상위 25 개 영화를 골라 60 번째 백분위 수 영화의 투표를 계산

그런 다음 이를 m 값으로 사용하여 Simple Recommender 섹션에서했던 것처럼 IMDB의 공식을 사용하여 각 영화의 가중치 등급을 계산합니다.

In [48]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.95)

In [45]:
def weighted_rating(x):
  v = x['vote_count']
  R = x['vote_average']
  return (v/(v+m) * R) + (m/(m+v) * C)

In [49]:
# 인기도와 평점을 고려한 향상된 추천

def improved_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]

  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
  vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
  C = vote_averages.mean()
  m = vote_counts.quantile(0.60)
  qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  qualified['wr'] = qualified.apply(weighted_rating, axis=1)
  qualified = qualified.sort_values('wr', ascending=False).head(10)
  return qualified

In [50]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.919065
8613,Interstellar,11187,8,2014,7.898936
6623,The Prestige,4510,8,2006,7.762198
3381,Memento,4168,8,2000,7.744491
8031,The Dark Knight Rises,9263,7,2012,6.922734
6218,Batman Begins,7511,7,2005,6.905676
1134,Batman Returns,1706,6,1992,5.848168
132,Batman Forever,1529,5,1995,5.051917
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013324
1260,Batman & Robin,1447,4,1997,4.281221


In [51]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.713637
390,Dazed and Confused,588,7,1993,6.261052
8883,The DUFF,1372,6,2015,5.819948
3712,The Princess Diaries,1063,6,2001,5.782558
4763,Freaky Friday,919,6,2003,5.759261
6277,Just Like Heaven,595,6,2005,5.682790
6959,The Spiderwick Chronicles,593,6,2008,5.682167
7494,American Pie Presents: The Book of Love,454,5,2009,5.115411
7332,Ghosts of Girlfriends Past,716,5,2009,5.088910
7905,Mr. Popper's Penguins,775,5,2011,5.084538


# **Collaborative Filtering**

In [52]:
reader = Reader()

In [53]:
ratings = pd.read_csv('/content/drive/MyDrive/moviedata/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [58]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'fit_time': (4.737488746643066,
  4.752794504165649,
  4.719443321228027,
  4.758547306060791,
  4.718016624450684),
 'test_mae': array([0.6902868 , 0.69332134, 0.6905018 , 0.68604049, 0.69506497]),
 'test_rmse': array([0.89240961, 0.90210074, 0.89729391, 0.895423  , 0.89935913]),
 'test_time': (0.3037443161010742,
  0.1832895278930664,
  0.1722567081451416,
  0.43048977851867676,
  0.1764969825744629)}

In [60]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [61]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [62]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.781345628744336, details={'was_impossible': False})

# **Hybrid Recommender**

In [63]:
# Contet_based + Collavorate 결합
# 입력 : 사용자 ID, 영화 제목
# 출력 : 특정 사용자의 예상 등급을 기준, 유사한 영화 정렬 추천

def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [64]:
id_map = pd.read_csv('/content/drive/MyDrive/moviedata/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [65]:
indices_map = id_map.set_index('id')

In [66]:
def hybrid(userId, title):
  idx = indices[title]
  tmdbId = id_map.loc[title]['id']
  movie_id = id_map.loc[title]['movieId']

  sim_scores = list(enumerate(cosine_sim[int(idx)]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]

  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
  movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
  movies = movies.sort_values('est', ascending=False)
  return movies.head(10)

In [67]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.080799
974,Aliens,3282.0,7.7,1986,679,3.067741
1011,The Terminator,4208.0,7.4,1984,218,3.037770
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.994165
922,The Abyss,822.0,7.1,1989,2756,2.932041
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,2.916145
2014,Fantastic Planet,140.0,7.6,1973,16306,2.755193
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.747702
4966,Hercules in New York,63.0,3.7,1969,5227,2.728457
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.707351


In [68]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.646944
4017,Hawk the Slayer,13.0,4.5,1980,25628,3.286098
974,Aliens,3282.0,7.7,1986,679,3.259663
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.253500
1011,The Terminator,4208.0,7.4,1984,218,3.227885
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.187111
4347,Piranha Part Two: The Spawning,41.0,3.9,1981,31646,3.170069
2014,Fantastic Planet,140.0,7.6,1973,16306,3.140872
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.126389
1376,Titanic,7770.0,7.5,1997,597,3.113792


# **결론**

서로 다른 아이디어와 알고리즘을 기반으로 4 개의 서로 다른 추천 엔진을 구축했습니다.

- Simple Recommender :이 시스템은 전체 TMDB 투표 수 및 투표 평균을 사용하여 일반적으로 특정 장르에 대한 인기 영화 차트를 작성했습니다. IMDB Weighted Rating System은 최종적으로 정렬이 수행 된 등급을 계산하는 데 사용되었습니다.

- Content_Based Filtering : 두 가지 콘텐츠 기반 엔진을 구축했습니다. 하나는 영화 개요와 태그 라인을 입력으로 사용하고 다른 하나는 출연진, 제작진, 장르 및 키워드와 같은 메타 데이터를 사용하여 예측을 제시했습니다. 우리는 또한 더 많은 투표와 더 높은 등급의 영화를 더 선호하도록 간단한 필터를 장치했습니다.

- Collaborative Filtering : 강력한 Surprise Library를 사용하여 단일 값 분해를 기반으로 협업 필터를 구축했습니다. 획득 한 RMSE는 1 미만이었고 엔진은 주어진 사용자 및 영화에 대한 예상 등급을 제공했습니다.

- Hybrid engine : 콘텐츠 및 협업 필터링에서 아이디어를 모아 해당 사용자에 대해 내부적으로 계산 한 예상 평점을 기반으로 특정 사용자에게 영화 제안을 제공하는 엔진을 구축했습니다.